### pyspark read kafka topic

In [8]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_data"
LIMIT = 10

In [16]:
BOOTSTRAP_SERVERS = "datafabric-kafka-kafka-bootstrap.kafka-farm.svc.cluster.local:9092"
TOPICS = 'test'
CONSUMER_GROUP = 'test-datafabric'

In [10]:
from pydatafabric.ye import get_spark
from pydatafabric.gcp import bq_to_df

extra_jars = ["kafka-clients-2.8.1.jar", "slf4j-api-1.7.30.jar", "lz4-java-1.7.1.jar", "snappy-java-1.1.8.1.jar", "spark-sql-kafka-0-10_2.12-3.1.2.jar", "unused-1.0.0.jar", "spark-token-provider-kafka-0-10_2.12-3.3.0.jar"]
extra_jars = ["gs://emart-datafabric-resources/jars/spark-kafka-jars/" + data for data in extra_jars]
extra_jars = ",".join(extra_jars)

spark = get_spark(extra_jars=extra_jars)

In [11]:
spark.conf.get('spark.jars')

'hdfs:///jars/spark-bigquery-with-dependencies_2.12-latest.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/kafka-clients-2.8.1.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/slf4j-api-1.7.30.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/lz4-java-1.7.1.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/snappy-java-1.1.8.1.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/spark-sql-kafka-0-10_2.12-3.1.2.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/unused-1.0.0.jar,gs://emart-datafabric-resources/jars/spark-kafka-jars/spark-token-provider-kafka-0-10_2.12-3.3.0.jar'

In [12]:
df = bq_to_df(f"""
    select review_id as key, comments as value
    from  `{PROJECT}.{DATASET}.{TABLE}`
    where comments != ''
    limit {LIMIT}
""", spark_session=spark)

In [13]:
df.show(10)

+----------------+----------------------------------+
|             key|                             value|
+----------------+----------------------------------+
|2208170324332372|      천도 복숭아  싱싱한게 좋아요|
|2208170545233407|    새콤하니 그럭저럭 먹을만 해요.|
|2208170024210318| 맛도 좋고 싱싱해서 좋았습니다....|
|2208171959293135|          맛잇아요 신선해서 조아요|
|2208170938372306|             달고 맛있어요........|
|2208172122122434|     골드키위 매일  하나씩 먹어요.|
|2208170326197242|         상큼한 포모사 자두 좋아요|
|2208171622152619|             자두 싱싱하고 달아요~|
|2208171215499613|요즘 비많이와서 사까마까하다 샀...|
|2208171137018485|                약간 단맛이 덜해요|
+----------------+----------------------------------+



In [14]:
df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



### Writing the output of Batch Queries to Kafka

In [15]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
  .option("topic", TOPICS) \
  .save()

22/10/13 14:55:37 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 2.0 (TID 2) (yellowelephant-w-1 executor 1): org.apache.kafka.common.KafkaException: Failed to construct kafka producer
	at org.apache.kafka.clients.producer.KafkaProducer.<init>(KafkaProducer.java:440)
	at org.apache.kafka.clients.producer.KafkaProducer.<init>(KafkaProducer.java:291)
	at org.apache.kafka.clients.producer.KafkaProducer.<init>(KafkaProducer.java:274)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool.createKafkaProducer(InternalKafkaProducerPool.scala:136)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool.$anonfun$acquire$1(InternalKafkaProducerPool.scala:83)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool.acquire(InternalKafkaProducerPool.scala:82)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool$.acquire(InternalKafkaProducerPool.scal

Py4JJavaError: An error occurred while calling o156.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 5) (yellowelephant-w-1 executor 1): org.apache.kafka.common.KafkaException: Failed to construct kafka producer
	at org.apache.kafka.clients.producer.KafkaProducer.<init>(KafkaProducer.java:440)
	at org.apache.kafka.clients.producer.KafkaProducer.<init>(KafkaProducer.java:291)
	at org.apache.kafka.clients.producer.KafkaProducer.<init>(KafkaProducer.java:274)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool.createKafkaProducer(InternalKafkaProducerPool.scala:136)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool.$anonfun$acquire$1(InternalKafkaProducerPool.scala:83)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.kafka010.producer.InternalKafkaProducerPool.acquire(InternalKafkaProducerPool.

### Reading Data from Kafka Data

In [ ]:
import json

starting_offsets = {TOPICS:{"0":1}} # TOPIC의 id가 0인 파티션의 1번째 Offsets의 데이터
starting_offsets = json.dumps(starting_offsets)


batch_df_reader = spark.read \
  .format("kafka") \
  .option("subscribe", TOPICS) \
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
  .option("kafka.security.protocol", "SSL") \
  .option("startingOffsets", starting_offsets)\
  .option("auto.offset.reset", "earliest") \
  .option("failOnDataLoss", "false") \
  .option("enable.auto.commit", "false")

batch_df = batch_df_reader.load()
batch_df = batch_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "topic", "partition", "offset", "timestamp")

In [ ]:
batch_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



### Creating a Kafka Source for Streaming Queries

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
  .option("subscribe", TOPICS) \
  .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

### Creating a Kafka Sink for Streaming Queries

In [ ]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
  .option("topic", TOPICS) \
  .start()

In [38]:
spark.stop()

### python kafka package

In [ ]:
!pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
from kafka import KafkaProducer
from json import dumps

producer = KafkaProducer(acks=0,
                         compression_type='gzip',
                         bootstrap_servers=BOOTSTRAP_SERVERS,
                         key_serializer=str.encode,
                         value_serializer=lambda x: dumps(x).encode('utf-8')
                         )

for i in range(1000):
    producer.send(TOPICS, key= 'key_' + str(i), value='value_' + str(i))
    producer.flush()

In [ ]:
from kafka import KafkaConsumer
from json import loads

consumer = KafkaConsumer(
    TOPICS,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id=CONSUMER_GROUP,    
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    consumer_timeout_ms=1000
)


# consumer 리스트를 가져온다
print('[begin] get consumer list')
for message in consumer:
    print("Topic: %s, Partition: %d, Offset: %d, Key: %s, Value: %s" % (
        message.topic, message.partition, message.offset, message.key, message.value
    ))
print('[end] get consumer list')

[begin] get consumer list
[end] get consumer list
